In [1]:
'''
README.md
* transfer the wav to spectrogram
'''

'\nREADME.md\n* transfer the wav to spectrogram\n'

In [2]:
import os
import librosa
import numpy as np

In [3]:
from config import MIN as config
SR = config.SR
N_FFT = config.N_FFT
HOP_LENGTH = config.HOP_LENGTH
IN_DIR = config.ORG_DIR
OUT_DIR = config.FIG_DIR
print('sr = {}\nn_fft = {}\nhop_length = {}'.format(SR, N_FFT, HOP_LENGTH))
print('in_dir = \'{}\'\nout_dir = \'{}\''.format(IN_DIR, OUT_DIR))

sr = 16000
n_fft = 512
hop_length = 256
in_dir = './WAV'
out_dir = './GRAPH'


In [4]:
# util.py

def duration (file):
    sr = config.SR
    length = len(file)
    minute = length/sr//60
    second = length//sr - 60*minute
    return minute, second

def get_timestamp (tick):
    if tick == 0:
        return 0
    return tick*config.Default.HOP_LENGTH - 384

#Get out_path for spectrograms
def get_out_path(file):
    out_path = OUT_DIR
    root = file.split('/')
    root.pop(0)
    root.pop(0)
    root[-1] = root[-1][:-4]
    out_path += '/' + root[0]
    if not os.path.isdir(out_path):
        os.mkdir(out_path)
        out_path += '/' + root[1]
        os.mkdir(out_path)
    else:
        out_path += '/' + root[1]
        if not os.path.isdir(out_path):
            os.mkdir(out_path)
    out_path += '/' + root[2]
    return out_path

#Get spectrograms
def wav2figs(file):
    y,sr = librosa.load(file, SR)
    D = librosa.stft(y, N_FFT, HOP_LENGTH, N_FFT)
    magnitude = np.abs(D)
    phase = np.angle(D)
    return magnitude, phase 

In [5]:
#process files
for root, _, files in os.walk(IN_DIR):
    if root.count('/') == 3:
        for filename in sorted(files):
            if not filename.startswith('.'):
                #get filenames
                file = root + '/' + filename
                
                #get magnitude and phase
                mag,phase = wav2figs(file)
                
                #save output
                out = get_out_path(file)
                np.save(out + '-m', mag)
                np.save(out + '-p', phase)

In [27]:
# #try convert a wav
# def figs2stft(mag, phs):
#     stft = mag * np.exp(1.j * phs)
#     return stft

# def stft2wav(stft):
#     y = librosa.istft(stft, HOP_LENGTH, N_FFT)
#     return y

# def write_wav(name, y): #mono wave
#     librosa.output.write_wav(OUT_DIR+'/'+name, y, SR)

# file = IN_DIR+'/Ruan/LiuBo/01ShiLinYeQu.wav'
# mag,phs = wav2figs(file)
# stft=figs2stft(mag,phs)
# y=stft2wav(stft)
# librosa.output.write_wav('./new.wav', y, SR)